In [1]:
import dace
import cupy as cp
import numpy as np
from IPython.display import Code
from typing import Optional

from dace import SDFG, properties
from dace.config import Config
from dace.transformation import pass_pipeline as ppl, transformation
from dace.sdfg import nodes
from dace import dtypes
from dace.transformation.passes.shared_memory_synchronization import DefaultSharedMemorySync
from dace.sdfg.state import LoopRegion, ConditionalBlock

## Weird Global To Global Example

This is actually a nice example which does not compile when using the legacy CUDACodeGen due to const checks.

In [2]:
sdfg = dace.SDFG.from_file('weird_global_to_global.sdfg')

In [4]:
#sdfg.compile()

In [5]:
#Code(sdfg.generate_code()[1].clean_code)

## Weir Shared To Shared Example

In [7]:
sdfg = dace.SDFG.from_file('weird_shared_to_shared_copy.sdfg')

In [9]:
Code(sdfg.generate_code()[1].clean_code)

/home/berkay/master-thesis/dace/dace/codegen/targets/cuda.py:1872: UserWarning: No `gpu_block_size` property specified on map "kernel_101". Falling back to the configuration entry `compiler.cuda.default_block_size`: 32,1,1. You can either specify the block size to use with the gpu_block_size property, or by adding nested `GPU_ThreadBlock` maps, which map work to individual threads. For more information, see https://spcldace.readthedocs.io/en/latest/optimization/gpu.html
  warnings.warn(


#include <cuda_runtime.h>
#include <dace/dace.h>


struct kernel_state_t {
    dace::cuda::Context *gpu_context;
};



DACE_EXPORTED int __dace_init_cuda(kernel_state_t *__state, int64_t N);
DACE_EXPORTED int __dace_exit_cuda(kernel_state_t *__state);



int __dace_init_cuda(kernel_state_t *__state, int64_t N) {
    int count;

    // Check that we are able to run cuda code
    if (cudaGetDeviceCount(&count) != cudaSuccess)
    {
        printf("ERROR: GPU drivers are not configured or cuda-capable device "
               "not found\n");
        return 1;
    }
    if (count == 0)
    {
        printf("ERROR: No cuda-capable devices found\n");
        return 2;
    }

    // Initialize cuda before we run the application
    float *dev_X;
    DACE_GPU_CHECK(cudaMalloc((void **) &dev_X, 1));
    DACE_GPU_CHECK(cudaFree(dev_X));

    

    __state->gpu_context = new dace::cuda::Context(1, 1);

    // Create cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamCreateWithFlags(&__state->gpu_context->internal_streams[i], cudaStreamNonBlocking));
        __state->gpu_context->streams[i] = __state->gpu_context->internal_streams[i]; // Allow for externals to modify streams
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventCreateWithFlags(&__state->gpu_context->events[i], cudaEventDisableTiming));
    }

    

    return 0;
}

int __dace_exit_cuda(kernel_state_t *__state) {
    

    // Synchronize and check for CUDA errors
    int __err = static_cast<int>(__state->gpu_context->lasterror);
    if (__err == 0)
        __err = static_cast<int>(cudaDeviceSynchronize());

    // Destroy cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamDestroy(__state->gpu_context->internal_streams[i]));
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventDestroy(__state->gpu_context->events[i]));
    }

    delete __state->gpu_context;
    return __err;
}

DACE_EXPORTED bool __dace_gpu_set_stream(kernel_state_t *__state, int streamid, gpuStream_t stream)
{
    if (streamid < 0 || streamid >= 1)
        return false;

    __state->gpu_context->streams[streamid] = stream;

    return true;
}

DACE_EXPORTED void __dace_gpu_set_all_streams(kernel_state_t *__state, gpuStream_t stream)
{
    for (int i = 0; i < 1; ++i)
        __state->gpu_context->streams[i] = stream;
}

__global__ void __launch_bounds__(32) kernel_101_0_0_0(const double * __restrict__ A, const double * __restrict__ B, double * __restrict__ C, int N) {
    {
        int i = (blockIdx.x * 32 + threadIdx.x);
        double __tmp1;
        __shared__ double shr_A[32];
        __shared__ double shr_B[32];
        if (i < N) {
            dace::GlobalToShared1D<double, 32, 1, 1, 1, 1, false>(A + i, 1, shr_A + (i % 32));
            dace::GlobalToShared1D<double, 32, 1, 1, 1, 1, false>(B + i, 1, shr_B + (i % 32));
            {
                double __in1 = shr_A[(i % 32)];
                double __in2 = shr_B[(i % 32)];
                double __out;

                ///////////////////
                // Tasklet code (_Add_)
                __out = (__in1 + __in2);
                ///////////////////

                __tmp1 = __out;
            }
            {
                double __inp = __tmp1;
                double __out;

                ///////////////////
                // Tasklet code (assign_102_16)
                __out = __inp;
                ///////////////////

                C[i] = __out;
            }
        }
    }
}


DACE_EXPORTED void __dace_runkernel_kernel_101_0_0_0(kernel_state_t *__state, const double * __restrict__ A, const double * __restrict__ B, double * __restrict__ C, int N);
void __dace_runkernel_kernel_101_0_0_0(kernel_state_t *__state, const double * __restrict__ A, const double * __restrict__ B, double * __restrict__ C, int N)
{

    if (((int_ceil(int_ceil(N, 1), 32)) <= 0)) {

        return;
    }

    void  *kernel_101_0_0_0_args[] = { (void